<a href="https://colab.research.google.com/github/Study001001/Google-Colab/blob/main/YTS_Torrent_%3E%3E_G_Drive_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This Script Supports YTS Torren. Use YTS [Torrent File] to Download ❤**  

### 1. Installing libraries

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

### 2. Mount your google drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### 3. Create session

In [ ]:
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)

downloads = [] # all torrent files and magnet links we want to download

### 4. Add torrent from file

In [ ]:
from google.colab import files

source = files.upload()

params = {
    "save_path": "/content/drive/Shareddrives/4 29 2022/MOVIE",
    "ti": lt.torrent_info(list(source.keys())[0])
}

downloads.append(ses.add_torrent(params))

### 5. Add torrent from magnet link

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, 
                          params)
    )

### 6.import time

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

### 7. Now Download :)

In [ ]:
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close()
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)